In [1]:
import pandas as pd  # Biblioteca para manipulación y análisis de datos. Ideal para trabajar con tablas.
import numpy as np  # Biblioteca para cálculos matemáticos complejos y manejo de arrays.

In [106]:
dataset = pd.read_csv('../data/raw/data.csv',sep=';')

In [107]:
target = 'ATTRITION'
X, y = dataset.drop([target, 'ID_CORRELATIVO', 'CODMES'], axis=1), dataset[[target]]

In [108]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,
                                                   y,
                                                   test_size=0.2,
                                                   random_state=1416,
                                                   stratify=y
                                                   )

In [109]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

imputer = ColumnTransformer(
    [
        ('mean_imputer', SimpleImputer(strategy='mean'), ['EDAD', 'ANTIGUEDAD']),
        ('mode_imputer', SimpleImputer(strategy='most_frequent'), ['RANG_INGRESO', 'FLAG_LIMA_PROVINCIA']),
        #('ohe', OneHotEncoder(categories='auto', drop=None, sparse_output=False, handle_unknown='error'), ['RANG_INGRESO','FLAG_LIMA_PROVINCIA','RANG_SDO_PASIVO_MENOS0','RANG_NRO_PRODUCTOS_MENOS0'])
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)

In [110]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

encoder = ColumnTransformer(
    [
        ('ohe', OneHotEncoder(categories='auto', drop=None, sparse_output=False, handle_unknown='error'), ['RANG_INGRESO','FLAG_LIMA_PROVINCIA','RANG_SDO_PASIVO_MENOS0','RANG_NRO_PRODUCTOS_MENOS0'])
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

X_train = encoder.fit_transform(X_train)
X_test = encoder.fit_transform(X_test)

### Selección de variables

In [ ]:
from sklearn.ensemble import RandomForestClassifier  # Modelo de ensamblaje basado en árboles de decisión para tareas de clasificación.

### Entreno un modelo de Random Forest con 200 arboles y una semilla aleatoria 123
rf_clf = RandomForestClassifier(n_estimators=200,random_state =123)   ## forma del modelo

# Entreno el modelo con el .fit
rf_clf.fit(X_train.values,y_train.values.ravel())
## ordenando  las mejores variables de mayor a menor generando un bucle en este caso un for para
## extraer los nombres de las columnas y la ganancia(Gain) de las variables mas impactantes

features = []
for feature in zip(X.columns, rf_clf.feature_importances_):
    features.append(feature)
    
## genero un dataframe para visualizar mejor lo que hizo random forest con su selector multivariado  
features_total = pd.DataFrame(features,columns=['Variables','Gain']).sort_values('Gain', ascending=False)
features_total

#### Selector de variables

In [ ]:
from sklearn.feature_selection import SelectFromModel  # Selector de características basado en importancias derivadas de modelos.

## uso SelectFromModel poniendo lo parametros del modelo que se contruyo y un threshold que es un punto de corte que
## puede asumir de acuerdo a tu criterio con respecto a las variables que estén generando mas Gain

sfm = SelectFromModel(rf_clf, threshold=0.008)

# entrenamiento del selector
sfm.fit(X, y)

# contruyo una lista para quedarme con las mejores variables

variables = [] #### lista vacía
for feature_list_index in sfm.get_support(indices=True): ## 
    variables.append(X.columns[feature_list_index])
    #variables
    
#features_total
variables

In [ ]:
best_X_train = X_train[variables]
best_X_test  = X_test[variables]

# Procesamiento

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score  # Funciones para calcular la precisión de clasificación y el área bajo la curva ROC.

## Modelos

### LightGBM

In [ ]:
from time import time
from lightgbm import LGBMClassifier  # Framework de boosting que utiliza algoritmos basados en árboles de decisión y es eficiente en grandes volúmenes de datos.

# construimos un lightgbm, ojo que para elY es el mismo porque es un vector univariado
best_model = LGBMClassifier() 
best_model.fit(best_X_train, y_train)


# predecimos el train y test con la probabilidad para validar luego
predict_train_lg = best_model.predict_proba(best_X_train)[:,1]
predict_test_lg = best_model.predict_proba(best_X_test)[:,1]


# imprimimos el roc de train y test con la data real y la prediccion de la probabilidad del modelo
print("auc o Roc on training in LGBMClassifier data : {:.3f}".format(roc_auc_score(y_train, predict_train_lg)))
print("auc o Roc on testing in LGBMClassifier  data : {:.3f}".format(roc_auc_score(y_test, predict_test_lg))) 

#### Optimización de hiperparámatros con validación cruzada 

In [ ]:
from pprint import pprint
# Mostramos todos los parametros que tiene ligthGBMlo usamos porque es mas rápido para tunear el modelo
print('Parameters currently in use:\n')
pprint(best_model.get_params()) 

In [ ]:
# Numero de arboles
n_estimators = [300, 400, 500]
# porcentaje de variables con la que se contruye un arbol
colsample_bytree = [0.7, 0.8, 0.9]
# profundidad del arbol
max_depth = [4, 6, 8]
# ratio de aprendizaje por cada arbol
learning_rate = [0.1, 0.15 ,0.2]
# creacion del grip search o grilla generando un diccionario
param_grid = {'n_estimators': n_estimators,
               'colsample_bytree': colsample_bytree,
               'max_depth': max_depth,
               'learning_rate': learning_rate}
pprint(param_grid)

In [ ]:
from sklearn.model_selection import GridSearchCV  # Herramientas para afinar hiperparámetros.

lgb = LGBMClassifier()
# Isntacia del grip search ponemos la grilla , la cantidad de kfolds para la validacion cruzada en este caso 5
# n_jobs -1 para la paralelizacion de la ejecucion en la optimizacion del modelo y verbose para mostrar de 2 en 2 los resultados
grid_search = GridSearchCV(estimator = lgb, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 2)
# FIT para ejecutarlo
X_best = X[variables] ## recordar que se usa toda la data completa y con las mejores variables
grid_search.fit(X_best, y)
grid_search.best_params_ # mostramos los mejores hiperparametros

#### Evaluación

In [ ]:
# Genero una funcion para validar la mejora de la curva ROC o auc para ver como aumenta el performance de mi modelo
def evaluate(model, test_features, test_labels):
    predictions = model.predict_proba(test_features)[:,1] ## recordar que para validar el roc se necesita el dato real y 
    roc = roc_auc_score(test_labels,predictions)          ## probabilidad de la prediccion
    print('Model Performance')
    print('roc_auc_score = {:.12g}%.'.format(roc))
    return roc

In [ ]:
base_model = LGBMClassifier()
base_model.fit(best_X_train, y_train)
base_roc = evaluate(base_model, best_X_test, y_test)

## pasamos la funcion evaluate para sacar el resultado del modelo
best_grid = grid_search.best_estimator_
grid_roc = evaluate(best_grid, best_X_test, y_test)
print('Mejora en {:.12g}%.'.format( 100 * (grid_roc - base_roc) / base_roc)) ## le damos diseño para imprimir con .format
## podemos visualizar el aumento o las mejora en 3.5%

In [ ]:
base_roc ## roc para el test  ---- modelo normal

In [ ]:
grid_roc ## roc para el test --- modelo optimizado

In [ ]:
base_model = LGBMClassifier()
base_model.fit(best_X_train, y_train)
base_roc = evaluate(base_model, best_X_train, y_train)

## pasamos la funcion evaluate para sacar el resultado del modelo
best_grid = grid_search.best_estimator_
grid_roc = evaluate(best_grid, best_X_train, y_train)
print('Mejora en {:.12g}%.'.format( 100 * (grid_roc - base_roc) / base_roc)) ## le damos diseño para imprimir con .format
## podemos visualizar el aumento o las mejora en 3.5%